<a href="https://colab.research.google.com/github/filipp134/vk_bot_detection/blob/forth_version/Telegram_bot_for_VK_bot_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install python-telegram-bot
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update, ParseMode
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackQueryHandler, CallbackContext, \
    ConversationHandler
import requests
BASIC_LINK = 'https://api.vk.com/method/'
ACCESS_TOKEN = '03b514fe3f199471fc77c727575320360cbd8895d5ac6fe464572df08e2d44b8a2ecb513a6f91184bc3bd'
TOKEN = '1796537740:AAETJ8c9xHgj6Xl62Fz4R8Obr_uaUF_smF0'

In [14]:
def get_user_info(user_id):
    '''Получение информации в виде словаря о пользователе ВКонтакте с user_id.'''
    method = 'users.get?user_ids={user_ids}&fields={fields}&access_token={access_token}&v={api_version}'
    payload = {
        'user_ids' : [user_id],
        'fields': ','.join(['city', 'sex', 'bdate', 'followers_count', 'has_mobile', 'personal']),
        'v':'5.130',
        'access_token' : ACCESS_TOKEN
    }
    response = requests.get(BASIC_LINK + method, params=payload).json()
    info = response['response'][0]
    return info

In [16]:
def start(update, context):
    """Начинаем общение с ботом"""
    context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")
    return ConversationHandler.END

In [17]:
def find_user_info(update, context):
    '''Выполняем поиск базовой информации по пользователю и отправляём её пользователю'''
    id = update.message.text
    result = '\n'.join([f'{k} : {v}' for k, v in get_user_info(id).items()])
    if not result:
        context.bot.send_message(chat_id=update.effective_chat.id, text="Пользователей с таким id не найдено!")
        return ConversationHandler.END
    context.bot.send_message(chat_id=update.effective_chat.id, text=result)
    return SEARCH_LIST

In [21]:
def main():
    updater = Updater(token=TOKEN, use_context=True)
    dispatcher = updater.dispatcher
    message_handler = MessageHandler(Filters.text & (~Filters.command), find_user_info)
    start_handler = CommandHandler('start', start)
    dispatcher.add_handler(message_handler)
    dispatcher.add_handler(start_handler)
    updater.start_polling()

In [22]:
main()